# Problem Statement

Task is to look at 60 DNA sequence elements (called "nucleotides" or "base-pairs") and decide if this is a 

a) "intron -> exon" boundary (ie) [These are sometimes called "donors"] 
b) "exon -> intron" boundary (ei) [These are sometimes called "acceptors"] 
c) neither

Data Source is here: https://archive.ics.uci.edu/ml/datasets/Molecular+Biology+%28Splice-junction+Gene+Sequences%29

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import re

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

#Import the dataset
data = pd.read_csv('data.csv')


#Take a look at the data 
data.describe()


Using TensorFlow backend.


,Class,Donor,Sequence
count,3190,3190,3190
unique,3,3178,3092
top,N,HUMMYLCA-ACCEPTOR-1831,TGACCTGATCTTTGCTCTCCCCCTGGCCAGTTGAGG...
freq,1655,2,3


In [9]:
#Let us look at a few entries
data.head()

,Class,Donor,Sequence
0,EI,ATRINS-DONOR-521,CCAGCTGCATCACAGGAGGCCAGCGAGCAGG...
1,EI,ATRINS-DONOR-905,AGACCCGCCGGGAGGCGGAGGACCTGCAGGG...
2,EI,BABAPOE-DONOR-30,GAGGTGAAGGACGTCCTTCCCCAGGAGCCGG...
3,EI,BABAPOE-DONOR-867,GGGCTGCGTTGCTGGTCACATTCCTGGCAGGT...
4,EI,BABAPOE-DONOR-2817,GCTCAGCCCCCAGGTCACCCAGGAACTGACGTG...


## Convert the target variable into an encoded class

In [6]:
#Create target variable with encoded class
y = pd.DataFrame(data['Class'])
cols_to_transform = y.dtypes[y.dtypes=="object"].index
y = pd.get_dummies(y,columns = cols_to_transform )

print(y.head())



   Class_EI  Class_IE  Class_N
0         1         0        0
1         1         0        0
2         1         0        0
3         1         0        0
4         1         0        0


## Convert the DNA sequence into an array of integers corresponding to unicode value of character

eg Sequence "CCAG.." would be an array [67,67,65,71...]

In [8]:
X = pd.DataFrame()

#Loop throgh all rows
#  - Convert char to unicode
for i in range (0,data.shape[0]):
    
    #Extract sequence
    seq = data.loc[i,'Sequence']
    
    #Remove whitespace
    seq = re.sub("\s+","", seq)
    
    #Conver char to unicode
    for j in range (0,len(seq)):
        X.loc[i,j] = ord(seq[j])

print(X.head())

     0     1     2     3     4     5     6     7     8     9   ...     50  \
0  67.0  67.0  65.0  71.0  67.0  84.0  71.0  67.0  65.0  84.0  ...   65.0   
1  65.0  71.0  65.0  67.0  67.0  67.0  71.0  67.0  67.0  71.0  ...   71.0   
2  71.0  65.0  71.0  71.0  84.0  71.0  65.0  65.0  71.0  71.0  ...   67.0   
3  71.0  71.0  71.0  67.0  84.0  71.0  67.0  71.0  84.0  84.0  ...   71.0   
4  71.0  67.0  84.0  67.0  65.0  71.0  67.0  67.0  67.0  67.0  ...   67.0   

     51    52    53    54    55    56    57    58    59  
0  71.0  67.0  67.0  65.0  71.0  84.0  67.0  84.0  71.0  
1  84.0  71.0  67.0  67.0  67.0  67.0  67.0  71.0  67.0  
2  65.0  67.0  71.0  71.0  71.0  71.0  65.0  84.0  71.0  
3  71.0  84.0  84.0  84.0  84.0  67.0  67.0  67.0  67.0  
4  67.0  84.0  84.0  71.0  65.0  67.0  67.0  67.0  84.0  

[5 rows x 60 columns]


## Split train and test/validation set. Also scale feature values 

In [7]:
from sklearn.model_selection import train_test_split
X_train_pre,X_val_pre,y_train,y_val =  train_test_split(X,y,test_size=0.2)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_pre=  sc.fit_transform(X_train_pre.values.astype(int))
X_val_pre=  sc.fit_transform(X_val_pre.values.astype(int))

y_train = y_train.values.astype(int)
y_val = y_val.values.astype(int)

# reshape input to be 3D for LSTM[samples, timesteps, features]
X_train = X_train_pre.reshape((X_train_pre.shape[0], 1, X_train_pre.shape[1]))
X_val =  X_val_pre.reshape((X_val_pre.shape[0], 1, X_val_pre.shape[1]))

C:\Users\takalyan\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int32 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


In [4]:
# fix random seed for reproducibility
np.random.seed(7)
epochs = 10

# design network
model = Sequential()
model.add(LSTM(1000, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(3, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())
model.fit(X_train, y_train, nb_epoch=epochs, batch_size=64)
y_val_pred = model.predict(X_val)

# Final evaluation of the model
scores = model.evaluate(X_val, y_val, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 1000)              4244000   
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 3003      
Total params: 4,247,003
Trainable params: 4,247,003
Non-trainable params: 0
_________________________________________________________________
None


C:\Users\takalyan\AppData\Local\Continuum\Anaconda3\lib\site-packages\keras\models.py:837: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/10
2552/2552 [==============================] - 14s - loss: 0.9211 - acc: 0.6740    
Epoch 2/10
2552/2552 [==============================] - 12s - loss: 0.6454 - acc: 0.7731    
Epoch 3/10
2552/2552 [==============================] - 13s - loss: 0.5163 - acc: 0.8033    
Epoch 4/10
2552/2552 [==============================] - 12s - loss: 0.4433 - acc: 0.8233    
Epoch 5/10
2552/2552 [==============================] - 14s - loss: 0.3859 - acc: 0.8437    
Epoch 6/10
2552/2552 [==============================] - 14s - loss: 0.3302 - acc: 0.8648    
Epoch 7/10
2552/2552 [==============================] - 12s - loss: 0.2760 - acc: 0.8938    
Epoch 8/10
2552/2552 [==============================] - 12s - loss: 0.2200 - acc: 0.9240    
Epoch 9/10
2552/2552 [==============================] - 12s - loss: 0.1745 - acc: 0.9385    
Epoch 10/10
2552/2552 [==============================] - 12s - loss: 0.1382 - acc: 0.9534    
Accuracy: 86.52%
